imoportando bibliotecas

In [51]:
import pandas as pd
import openpyxl
import awswrangler as awr
import sqlite3

gerando dfs

In [40]:
read_path = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\CAMPANHA_RANKING_ATIVACOES.xlsx"

df_ativ = pd.read_excel(read_path, engine='openpyxl', sheet_name='ATIVAÇÕES')
df_cancel = pd.read_excel(read_path, engine='openpyxl', sheet_name='CANCELAMENTOS')


query_athena = """
SELECT parent, id_set, historic, note 
FROM silver.insurance_reg_historic

UNION ALL

SELECT parent, id_set, historic, note
FROM stcoop.insurance_reg_historic

UNION ALL

SELECT parent, id_set, historic, note
FROM viavante.insurance_reg_historic
"""

df_historico = awr.athena.read_sql_query(query_athena, database='silver')


criando dfs filtrados

In [52]:
#setando para datetime com pandas
df_cancel['data_cancelamento']=pd.to_datetime(df_cancel['data_cancelamento'])

df_ativ['data_ativacao'] = pd.to_datetime(df_ativ['data_ativacao'])


#filtrando datas
df_cancel_1trim = df_cancel[df_cancel['data_cancelamento'].between('2025-01-01', '2025-03-31')]

df_ativ_2trim = df_ativ[df_ativ['data_ativacao'].dt.month == 4]

#filtrando empresas

df_cancel_1trim_seg_st = df_cancel_1trim[df_cancel_1trim['empresa'].isin(['Segtruck','Stcoop'])]

df_ativ_2trim_via = df_ativ_2trim[df_ativ_2trim['empresa']=='Viavante']


#simplificando nomes
df_cancel = df_cancel_1trim_seg_st

df_ativ = df_ativ_2trim_via




conectando com sqlite3

In [53]:
import sqlite3

conn  = sqlite3.connect(":memory:") #criando banco temporário

df_cancel.to_sql("canceladas", conn, index=False, if_exists='replace')

df_ativ.to_sql("ativadas", conn, index=False, if_exists='replace')

df_historico.to_sql("historico", conn, index=False, if_exists='replace')





1058647

query

In [57]:
query = """
SELECT 
    c.placa,
    c.chassi,
    c.id_placa,
    c.id_veiculo,
    c.id_carroceria,

    c.matricula AS matricula_cancelamento,
    c.conjunto AS conjunto_cancelamento,
    c.unidade AS unidade_cancelamento, 
    c.data_cancelamento AS data_cancelamento,
    c.empresa AS empresa_cancelamento,

    a.matricula AS matricula_ativacao,
    a.conjunto AS conjunto_ativacao,
    a.unidade AS unidade_ativacao, 
    a.data_ativacao AS data_ativacao,
    a.empresa AS empresa_ativacao,
    
    h.historic,
    h.note
    
FROM canceladas AS c
INNER JOIN ativadas a ON a.placa = c.placa
LEFT JOIN historico h ON h.id_set = a.conjunto AND h.parent = a.matricula 

WHERE h.historic LIKE "%MIGRA%" OR
h.note LIKE "%MIGRA%"
"""

df_cancel_ativ = pd.read_sql_query(query,conn)

df_cancel_ativ['data_ativacao'] = pd.to_datetime(df_cancel_ativ['data_ativacao'])
df_cancel_ativ['data_ativacao'] = df_cancel_ativ['data_ativacao'].dt.date

df_cancel_ativ['data_cancelamento'] = pd.to_datetime(df_cancel_ativ['data_cancelamento'])
df_cancel_ativ['data_cancelamento'] = df_cancel_ativ['data_cancelamento'].dt.date

df_cancel_ativ.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula_cancelamento,conjunto_cancelamento,unidade_cancelamento,data_cancelamento,empresa_cancelamento,matricula_ativacao,conjunto_ativacao,unidade_ativacao,data_ativacao,empresa_ativacao,historic,note


verificação de meses

In [55]:
df_cancel_ativ['data_cancelamento'] = pd.to_datetime(df_cancel_ativ['data_cancelamento'])
df_cancel_ativ['data_ativacao'] = pd.to_datetime(df_cancel_ativ['data_ativacao'])

meses_cancel = df_cancel_ativ['data_cancelamento'].dt.month.unique()
meses_ativ = df_cancel_ativ['data_ativacao'].dt.month.unique()

print(meses_cancel)
print(meses_ativ)

[1 2 3]
[4]


gerando excel final

In [58]:
save_path = r"C:\Users\raphael.almeida\Documents\Projetos\Placas Canceladas Ativadas\placas_canceladas_ativadas.xlsx"
df_cancel_ativ.to_excel(save_path, engine='openpyxl', index=False, sheet_name="placas_canceladas_ativadas")